In [ ]:
# ===========================================
# 셀 3: 데이터 로드
DATA_PATH = "/content/data/" # Corrected typo from '/condent/data/'
RESULT_PATH = "/content/prediction/"

train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
val_df = pd.read_csv(os.path.join(DATA_PATH, 'dev.csv'))

In [ ]:
# ===========================================
# 셀 4: 카테고리별 Few-shot 샘플 설정

# 토픽을 상위 카테고리로 그룹핑하는 함수
def categorize_topic(topic):
    topic = topic.lower() if isinstance(topic, str) else ""

    # 음식/주문 관련
    if any(kw in topic for kw in ['음식', '주문', '배달', '식당', '메뉴', '식사', '요리', '카페', '커피']):
        return 'food_order'

    # 면접/취업 관련
    if any(kw in topic for kw in ['면접', '취업', '채용', '이력서', '자기소개']):
        return 'interview'

    # 여행/예약 관련
    if any(kw in topic for kw in ['여행', '호텔', '예약', '항공', '숙소', '체크인', '관광', '휴가']):
        return 'travel'

    # 쇼핑/거래 관련
    if any(kw in topic for kw in ['쇼핑', '구매', '가격', '할인', '환불', '상품', '임대']):
        return 'shopping'

    # 의료 관련
    if any(kw in topic for kw in ['의료', '병원', '진료', '증상', '건강', '약']):
        return 'medical'

    # 길/교통 관련
    if any(kw in topic for kw in ['길', '택시', '교통', '버스', '지하철', '운전', '주행']):
        return 'transport'

    # 일상/기타
    return 'daily'

# train_df에 category 컬럼 추가
train_df['category'] = train_df['topic'].apply(categorize_topic)
val_df['category'] = val_df['topic'].apply(categorize_topic)

# 카테고리별 few-shot 샘플 딕셔너리 생성
category_samples = {}
for cat in train_df['category'].unique():
    cat_df = train_df[train_df['category'] == cat]
    samples = cat_df.sample(min(2, len(cat_df)), random_state=42)
    category_samples[cat] = {
        'dialogue1': samples.iloc[0]['dialogue'],
        'summary1': samples.iloc[0]['summary'],
        'dialogue2': samples.iloc[1]['dialogue'] if len(samples) > 1 else samples.iloc[0]['dialogue'],
        'summary2': samples.iloc[1]['summary'] if len(samples) > 1 else samples.iloc[0]['summary'],
    }

print("카테고리별 few-shot 샘플 준비 완료!")
print(f"카테고리 목록: {list(category_samples.keys())}")

In [ ]:
print('--- train_df head ---')
display(train_df.head())
print('\n--- val_df head ---')
display(val_df.head())

print('\n--- train_df info ---')
train_df.info()
print('\n--- val_df info ---')
val_df.info()

In [ ]:
train_df['topic'].value_counts()

In [ ]:
# 토픽 분포 전체 확인
print("=" * 50)
print("토픽 분포 (상위 20개)")
print("=" * 50)
topic_counts = train_df['topic'].value_counts()
print(topic_counts.head(20))

print(f"\n총 고유 토픽 수: {train_df['topic'].nunique()}")

# 토픽별 샘플 수 분포
print("\n" + "=" * 50)
print("토픽별 샘플 수 통계")
print("=" * 50)
print(topic_counts.describe())

# 샘플 수 적은 토픽들
print("\n샘플 수 10개 이하 토픽:")
print(topic_counts[topic_counts <= 10])

In [ ]:
# 토픽을 상위 카테고리로 그룹핑
def categorize_topic(topic):
    topic = topic.lower() if isinstance(topic, str) else ""

    # 음식/주문 관련
    if any(kw in topic for kw in ['음식', '주문', '배달', '식당', '메뉴', '식사', '요리', '카페', '커피']):
        return 'food_order'

    # 면접/취업 관련
    if any(kw in topic for kw in ['면접', '취업', '채용', '이력서', '자기소개']):
        return 'interview'

    # 여행/예약 관련
    if any(kw in topic for kw in ['여행', '호텔', '예약', '항공', '숙소', '체크인', '관광', '휴가']):
        return 'travel'

    # 쇼핑/거래 관련
    if any(kw in topic for kw in ['쇼핑', '구매', '가격', '할인', '환불', '상품', '임대']):
        return 'shopping'

    # 의료 관련
    if any(kw in topic for kw in ['의료', '병원', '진료', '증상', '건강', '약']):
        return 'medical'

    # 길/교통 관련
    if any(kw in topic for kw in ['길', '택시', '교통', '버스', '지하철', '운전', '주행']):
        return 'transport'

    # 일상/기타
    return 'daily'

train_df['category'] = train_df['topic'].apply(categorize_topic)

print("=" * 50)
print("카테고리별 분포")
print("=" * 50)
print(train_df['category'].value_counts())

# 카테고리별 예시 토픽 확인
print("\n" + "=" * 50)
print("카테고리별 토픽 예시")
print("=" * 50)
for cat in train_df['category'].unique():
    topics = train_df[train_df['category'] == cat]['topic'].value_counts().head(5)
    print(f"\n[{cat}]")
    print(topics)

In [ ]:
# 각 카테고리별 샘플 확인 (dialogue, summary 구조 파악)
print("=" * 50)
print("데이터 컬럼 확인")
print("=" * 50)
print(train_df.columns.tolist())

# 첫 번째 샘플로 dialogue/summary 필드명 확인
print("\n첫 번째 샘플 키:")
print(train_df.iloc[0].to_dict().keys())

In [ ]:
# 카테고리별 샘플 예시 확인 (dialogue, summary 길이 포함)
# 필드명은 실제 데이터에 맞게 수정 필요

dialogue_col = 'dialogue'  # 실제 컬럼명으로 수정
summary_col = 'summary'    # 실제 컬럼명으로 수정

for cat in ['food_order', 'interview', 'travel', 'shopping', 'medical', 'transport', 'daily']:
    print("=" * 60)
    print(f"[{cat}] 샘플 예시")
    print("=" * 60)

    sample = train_df[train_df['category'] == cat].iloc[0]

    print(f"\n토픽: {sample['topic']}")
    print(f"\n대화:\n{sample[dialogue_col][:500]}...")
    print(f"\n요약:\n{sample[summary_col]}")
    print(f"\n대화 길이: {len(sample[dialogue_col])}, 요약 길이: {len(sample[summary_col])}")
    print("\n")

### 2. 결측치 확인

In [ ]:
print('--- train_df missing values ---')
print(train_df.isnull().sum())
print('\n--- val_df missing values ---')
print(val_df.isnull().sum())

### 3. 대화 및 요약 길이 분포 분석

대화와 요약문의 단어 및 글자 수 분포를 확인하여 데이터의 특성을 파악합니다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 대화 길이 (글자 수)
train_df['dialogue_len_char'] = train_df['dialogue'].apply(len)
val_df['dialogue_len_char'] = val_df['dialogue'].apply(len)

# 요약 길이 (글자 수)
train_df['summary_len_char'] = train_df['summary'].apply(len)
val_df['summary_len_char'] = val_df['summary'].apply(len)

# 대화 길이 (단어 수)
train_df['dialogue_len_word'] = train_df['dialogue'].apply(lambda x: len(x.split()))
val_df['dialogue_len_word'] = val_df['dialogue'].apply(lambda x: len(x.split()))

# 요약 길이 (단어 수)
train_df['summary_len_word'] = train_df['summary'].apply(lambda x: len(x.split()))
val_df['summary_len_word'] = val_df['summary'].apply(lambda x: len(x.split()))

plt.figure(figsize=(18, 12))

plt.subplot(2, 2, 1)
sns.histplot(train_df['dialogue_len_char'], bins=50, kde=True)
plt.title('Train Dialogue Length (Characters)')

plt.subplot(2, 2, 2)
sns.histplot(train_df['summary_len_char'], bins=50, kde=True)
plt.title('Train Summary Length (Characters)')

plt.subplot(2, 2, 3)
sns.histplot(train_df['dialogue_len_word'], bins=50, kde=True)
plt.title('Train Dialogue Length (Words)')

plt.subplot(2, 2, 4)
sns.histplot(train_df['summary_len_word'], bins=50, kde=True)
plt.title('Train Summary Length (Words)')

plt.tight_layout()
plt.show()

print('\n--- Train Dialogue Length (Characters) Statistics ---')
print(train_df['dialogue_len_char'].describe())
print('\n--- Train Summary Length (Characters) Statistics ---')
print(train_df['summary_len_char'].describe())
print('\n--- Train Dialogue Length (Words) Statistics ---')
print(train_df['dialogue_len_word'].describe())
print('\n--- Train Summary Length (Words) Statistics ---')
print(train_df['summary_len_word'].describe())

In [2]:
# AI Hub JSON 데이터 구조 확인

import json
import os

# JSON 파일 경로
data_dir = '/Users/seoyeonmun/nlp_project/korean_data/train_15000'

# 폴더 내 모든 JSON 파일 확인
json_files = [f for f in os.listdir(data_dir) if f.endswith('.json')]
print(f"JSON 파일 목록 ({len(json_files)}개):")
for f in json_files:
    print(f"  - {f}")

print("\n" + "=" * 50)

# 각 JSON 파일의 구조 확인
for json_file in json_files:
    json_path = os.path.join(data_dir, json_file)
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"\n📁 {json_file}")
    print(f"  타입: {type(data)}")

    if isinstance(data, list):
        print(f"  개수: {len(data)}개")
        print(f"\n  첫 번째 샘플:")
        print(json.dumps(data[0], indent=2, ensure_ascii=False)[:1000])
    
    elif isinstance(data, dict):
        print(f"  키: {list(data.keys())}")
        
        # 'data' 키가 있으면 해당 내용 확인
        if 'data' in data:
            print(f"  numberOfItems: {data.get('numberOfItems', 'N/A')}")
            print(f"  data 개수: {len(data['data'])}개")
            print(f"\n  첫 번째 샘플:")
            print(json.dumps(data['data'][0], indent=2, ensure_ascii=False)[:2000])
        else:
            first_key = list(data.keys())[0]
            print(f"\n  첫 번째 키({first_key}) 내용:")
            content = data[first_key][:1] if isinstance(data[first_key], list) else data[first_key]
            print(json.dumps(content, indent=2, ensure_ascii=False)[:2000])
    
    print("\n" + "=" * 50)
    break  # 첫 번째 파일만 상세 확인 (전체 보려면 이 줄

JSON 파일 목록 (9개):
  - 상거래(쇼핑).json
  - 식음료.json
  - 미용과건강.json
  - 여가생활.json
  - 시사교육.json
  - 일과직업.json
  - 개인및관계.json
  - 행사.json
  - 주거와생활.json


📁 상거래(쇼핑).json
  타입: <class 'dict'>
  키: ['numberOfItems', 'data']
  numberOfItems: 1666
  data 개수: 1666개

  첫 번째 샘플:
{
  "header": {
    "dialogueInfo": {
      "dialogueID": "34e45d83-41ea-5321-a983-bc257da2848f",
      "numberOfParticipants": 2,
      "numberOfUtterances": 16,
      "numberOfTurns": 8,
      "type": "일상 대화",
      "topic": "상거래(쇼핑)"
    },
    "participantsInfo": [
      {
        "participantID": "P01",
        "gender": "여성",
        "age": "20대",
        "residentialProvince": "전라남도"
      },
      {
        "participantID": "P02",
        "gender": "여성",
        "age": "20대",
        "residentialProvince": "전라남도"
      }
    ]
  },
  "body": {
    "dialogue": [
      {
        "utteranceID": "U1",
        "turnID": "T1",
        "participantID": "P01",
        "date": "2015-04-02",
        "time": "12:52:00",
       